In [13]:
import sys
import os
import shutil

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import pyvista as pv
import cmcrameri.cm as cmc
import cv2

from tqdm import tqdm

import warnings
warnings.filterwarnings("ignore", category=UserWarning)

from matplotlib import rc
rc("pdf", fonttype=42)

from gdtchron import plot

In [2]:
os.environ['GALLIUM_DRIVER']='softpipe'
pv.start_xvfb()

In [3]:
# Get full output files from local source
sol_path = '/home/vaseylab/glaucophane/fastscape_testing/060524_rif/output_ri_rift/solution'
files = [os.path.join(sol_path,file) for file in os.listdir(sol_path) if file.endswith('.pvtu')]
files.sort()

In [4]:
directory_He = 'meshes_tchron'
time1 = 16
time2 = 36

model_step = 0.1

step1 = int(time1/model_step)
step2 = int(time2/model_step)

file1 = files[step1]
file2 = files[step2]

file1_He = os.path.join(directory_He,'meshes_tchron_160.vtu')
file2_He = os.path.join(directory_He,'meshes_tchron_360.vtu')

colors=['#66CCEE','#BBBBBB','#EE6677','#228833']
cm = ListedColormap(colors)

opaque_cm = cmc.lapaz_r
cm_strain = 'inferno_r'
cat_cmap = cmc.batlowS

opacity_strain = [0,0.8,0.8,0.8,0.8]
lim_strain = [0,5]

lim_He = [0,36]

bounds = [250,750,450,620]

In [ ]:
fig,axs = plt.subplots(3,2,dpi=300,figsize=(7,5))

axs = axs.flatten()

axs[0].set_title('Rift - 16 Myr',fontsize=10)
axs[1].set_title('Orogen - 36 Myr',fontsize=10)

axs[2].set_title('AHe Ages',loc='center',fontsize=10)

plot.plot2D(file1,'comp_field',bounds=bounds,ax=axs[0],
           cmap=cm,colorbar=False,contours=True)

plot.plot2D(file2,'comp_field',bounds=bounds,ax=axs[1],
            cmap=cm,colorbar=False,contours=True)

plot.plot2D(file1,'noninitial_plastic_strain',bounds=bounds,ax=axs[0],
          cmap=cm_strain,opacity=opacity_strain,clim=lim_strain)

plot.plot2D(file2,'noninitial_plastic_strain',bounds=bounds,ax=axs[1],
          cmap=cm_strain,opacity=opacity_strain,clim=lim_strain)

plot.plot2D(file1_He,'AHe',bounds=bounds,ax=axs[2],
            cmap=opaque_cm,colorbar=False,clim=lim_He)

plot.plot2D(file2_He,'AHe',bounds=bounds,ax=axs[3],
            cmap=opaque_cm,colorbar=False,clim=lim_He)

mesh1 = pv.read(file1_He)
mesh2 = pv.read(file2_He)

x1 = np.round(mesh1.points[:,0]/1000,0)
x2 = np.round(mesh2.points[:,0]/1000,0)
He1 = mesh1['AHe']
He2 = mesh2['AHe']

AFT1 = mesh1['AFT']
AFT2 = mesh2['AFT']

ZHe1 = mesh1['ZHe']
ZHe2 = mesh2['ZHe']

df1 = pd.DataFrame({'x':x1,'AHe':He1,'AFT':AFT1,'ZHe':ZHe1})
df1_max = df1.groupby('x').agg({'x':'first','AHe':'max','AFT':'max','ZHe':'max'})
df2 = pd.DataFrame({'x':x2,'AHe':He2,'AFT':AFT2,'ZHe':ZHe2})
df2_max = df2.groupby('x').agg({'x':'first','AHe':'max','AFT':'max','ZHe':'max'})

axs[0].set_ylabel('Y Distance (km)',fontsize=6)
axs[2].set_ylabel('Y Distance (km)',fontsize=6)

axs[4].plot(df1_max['x'],df1_max['AHe'],c=cat_cmap.colors[6],label='AHe')
axs[4].plot(df1_max['x'],df1_max['AFT'],c=cat_cmap.colors[4],label='AFT')
axs[4].plot(df1_max['x'],df1_max['ZHe'],c=cat_cmap.colors[2],label='ZHe')

axs[4].legend(fontsize=8)

axs[5].plot(df2_max['x'],df2_max['AHe'],c=cat_cmap.colors[6])
axs[5].plot(df2_max['x'],df2_max['AFT'],c=cat_cmap.colors[4])
axs[5].plot(df2_max['x'],df2_max['ZHe'],c=cat_cmap.colors[2])

axs[4].set_xlim(250,750)
axs[5].set_xlim(250,750)
axs[4].set_ylim(40,-5)
axs[5].set_ylim(40,-5)
axs[4].set_title('Surface Ages',fontsize=10)
axs[4].set_ylabel('Maximum Age (Ma)',fontsize=6)
axs[4].set_xlabel('X Distance (km)',fontsize=6)
axs[5].set_xlabel('X Distance (km)',fontsize=6)

for ax in axs:
    ax.tick_params(axis='both', which='major', labelsize=6)

plt.tight_layout()

fig.savefig('riftvsinv.pdf')
fig.savefig('riftvsinv.jpg')

In [16]:
# Create video - image creation
image_dir = 'images/'
shutil.rmtree(image_dir)
try:
    shutil.rmtree(image_dir)
except:
    print("Creating new image directory...")
else:
    print("Cleared existing image directory...")

os.makedirs(image_dir,exist_ok=False)

for k,file in enumerate(tqdm(files)):
    fig = plt.figure(dpi=300,figsize=(6.5,4))
    
    ax0 = fig.add_subplot(221)
    ax1 = fig.add_subplot(222)
    ax2 = fig.add_subplot(212)
    axs = [ax0,ax1,ax2]
    
    he_file = os.path.join(directory_He,'meshes_tchron_'+ str(k).zfill(3) +'.vtu')
    
    time = str(round(k*model_step,1)) + ' Ma'
    time_file = str(round(k*model_step,1)).zfill(4).replace('.','-')

    axs[0].set_title(time,loc='left')

    plot.plot2D(file,'comp_field',bounds=bounds,ax=axs[0],
               cmap=cm,colorbar=False,contours=True)
    
    plot.plot2D(file,'noninitial_plastic_strain',bounds=bounds,ax=axs[0],
              cmap=cm_strain,opacity=opacity_strain,clim=lim_strain)    

    plot.plot2D(he_file,'AHe',bounds=bounds,ax=axs[1],
            cmap=opaque_cm,colorbar=False,clim=lim_He)

    mesh = pv.read(he_file)
    x = np.round(mesh.points[:,0]/1000,0)
    AHe = mesh['AHe']
    AFT = mesh['AFT']
    ZHe = mesh['ZHe']

    df = pd.DataFrame({'x':x,'AHe':AHe,'AFT':AFT,'ZHe':ZHe})
    df_max = df.groupby('x').agg({'x':'first','AHe':'max','AFT':'max','ZHe':'max'})
    
    axs[2].plot(df_max['x'],df_max['AHe'],c=cat_cmap.colors[6],label='AHe')
    axs[2].plot(df_max['x'],df_max['AFT'],c=cat_cmap.colors[4],label='AFT')
    axs[2].plot(df_max['x'],df_max['ZHe'],c=cat_cmap.colors[2],label='ZHe')

    axs[0].set_ylabel('Y Distance (km)',fontsize=6)

    axs[1].set_title('AHe')

    cax_ahe = plot.add_colorbar(fig,cmap=opaque_cm,vmin=0,vmax=36,location=[0.7,0.72,0.15,0.02])
    cax_ahe.set_xticks([0,36])
    cax_ahe.tick_params(axis='both', which='major', labelsize=6)
    cax_ahe.set_title('AHe Age (Ma)',fontsize=6,pad=2)

    axs[2].set_xlim(250,750)
    axs[2].set_ylim(40,-2)
    axs[2].set_title('Surface Ages',fontsize=10)
    axs[2].set_ylabel('Maximum Age (Ma)',fontsize=6)
    axs[2].legend(fontsize=6)

    for ax in axs:
        ax.tick_params(axis='both', which='major', labelsize=6)
        ax.set_xlabel('X Distance (km)',fontsize=6)
    
    plt.tight_layout()
    fig.savefig(image_dir+time_file+'.jpg')
    plt.close()  

Creating new image directory...


100%|███████████████████████████████████████████████████████████████████████████████| 361/361 [1:02:03<00:00, 10.31s/it]


In [17]:
# Make movie
img_paths = [image_dir+file for file in sorted(os.listdir(image_dir)) if file.endswith('.jpg')]

frame = cv2.imread(img_paths[0])
height,width,layers = frame.shape

fourcc = cv2.VideoWriter_fourcc(*'avc1')

frate = 1/model_step

video = cv2.VideoWriter('video_riftinversion.mp4', fourcc, frate, (width,height))

for img in img_paths:
    video.write(cv2.imread(img))

cv2.destroyAllWindows()
video.release()

In [18]:
from IPython.display import Video

video_path = 'video_riftinversion.mp4'
Video(video_path, embed=True, width=800)